In [2]:
whos

Interactive namespace is empty.


In [441]:
#!pip install selenium
#!pip install bs4
#!pip install fnmatch

In [170]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import InvalidElementStateException
import time
import pprint
import re
import csv
import fnmatch
import os
import numpy as np
import pickle
from random import choice
from pprint import pprint

In [53]:
BASE_URL = 'https://scholar.google.com/citations'

In [2]:
desktop_agents = ['Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/602.2.14 (KHTML, like Gecko) Version/10.0.1 Safari/602.2.14',
                 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36',
                 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.98 Safari/537.36',
                 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.98 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0']
 
def random_headers():
    return {'User-Agent': choice(desktop_agents),'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'}
 

In [3]:
def get_users(): 
    
    #links from spreadsheet, next time we can grab from wixs in future
    sla_authors = ['https://scholar.google.com/citations?user=v9rIqPQAAAAJ&hl=en',
    'https://scholar.google.com/citations?hl=en&user=igIKOZcAAAAJ&view_op=list_works&sortby=pubdate',
    'https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=timothy+malloy+ucla&btnG=',
    'https://scholar.google.com/citations?user=gkygJ18AAAAJ&hl=en&oi=ao',
    'https://scholar.google.com/citations?user=1AG9uzsAAAAJ&hl=en',
    'https://scholar.google.com/citations?user=Rl5xhgwAAAAJ&hl=en&oi=ao',
    'https://scholar.google.com/citations?user=AfHMFccAAAAJ&hl=en&oi=ao',
    'https://scholar.google.com/citations?user=VXvek_AAAAAJ&hl=en&oi=sra',
    'https://scholar.google.com/citations?user=9CC9gHoAAAAJ&hl=en&oi=ao',
    'https://scholar.google.com/citations?user=rOvEiAYAAAAJ&hl=en',
    'https://scholar.google.com/citations?user=GVYNtNsAAAAJ&hl=en',
    'https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=CW+Wong+ucla&btnG=',
    'https://scholar.google.com/citations?user=ceCfTvcAAAAJ&hl=en&oi=ao',
    'https://scholar.google.com/citations?user=kvSIKM8AAAAJ&hl=en&oi=ao']

    user_id_regex = re.compile("user=(.{12})&")

    user_id = [m.group(1) for a in sla_authors for m in [user_id_regex.search(a)] if m]
    #[au in au for sla_authors]

    #print(user_id)
    print(len(user_id))
    
    return user_id

In [4]:
def get_full_user_page(user):
    """
    Will click to show all of the articles.
    
    :param user: an ID for a google scholar user
    :returns: returns a list of articles
    
    """
    
    url = BASE_URL + "?user=" + user
    
    driver.get(url)

    pages_remaining = True

    while pages_remaining:

        try:
            #Checks if there are more pages with links
            driver.find_element_by_id('gsc_bpf_more').click()         
            time.sleep((10-5)*np.random.random()+5) #from 5 to 30 seconds
            #print('pages_remaining ', pages_remaining)
        except InvalidElementStateException:
            gscholar_citations = driver.page_source
            pages_remaining = False
            #print('pages_remaining ', pages_remaining)
    
    #driver.quit()s
    
    return gscholar_citations


In [5]:
def get_author_citations_links(page):
    """
    Will take a google sholar profile page and convert to BeautifulSoup object, 
    then extract the author, uid, cid, title and citation page ID. 
    Then returns as list of dictionaries. 
    
    :param page: a google scholar author page
    :returns: returns a list of dicts that contain he author, uid, cid, 
    title and citation page ID
    """
    #make a soup object
    gscholar_soup = BeautifulSoup(page, 'lxml')
    
    citations = {}
    author = gscholar_soup.find(id="gsc_prf_in").get_text()
    for cite in gscholar_soup.select('td.gsc_a_t:
        citation = {}
        citation['title'] = cite.find(id="").get_text()
        #get the IDs from the author_cite_lists 
        #TODO: need to alter below to extract pattern v9rIqPQAAAAJ:u5HHmVD_uO8C
        #TODO: need to split on : v9rIqPQAAAAJ:u5HHmVD_uO8C and assign first part 
        # to uid and second to cid 
        citation_id_regex = re.compile("citation_for_view=(.{25})")
        citation_ids = [m.group(1) for c in citation_links for m in [citation_id_regex.search(c)] if m]

        citation['uid'] = 'v9rIqPQAAAAJ'
        citation['cid'] = 'u5HHmVD_uO8C'
         #title and citation page ID        
    
    return citations

In [4]:
def get_external_publisher_link(author_cids):
    """
    Takes a list that contains the author and her citation page links. Opens and scrapes
    citation pages and builds a dictionary of the author and citations. Returns the dict.
    
    :param links: a list containing author and another list of citation pages.
    :returns: returns a dictionary with the author and their citations.
    """
    
    #print("Author: ", author_cids[0][0], len(cids))
    
    author_bib = []
    
    for author, cid in author_cids: 
        for c in cid:
            print(author,c)
            #citation_for_view=igIKOZcAAAAJ:tntj4plCNvAC
            search_params = {'view_op': 'view_citation',
                             'h1': 'en',
                             'citation_for_view':c}
            cite_page = requests.get(BASE_URL, headers=random_headers(), params=search_params)
            print(cite_page.url)
            cpage_soup = BeautifulSoup(cite_page.text, 'lxml')
            #if cpage_soup.select_one('div#gsc_ocd_view').text in ['Investors', 'Conference']:
            #    continue 
            #else:
            citation = {}
            citation['title'] = cpage_soup.select('div.gsc_vcd_merged_snippet div a')[0].text
            citation['url'] = cpage_soup.select('div.gsc_vcd_merged_snippet div a')[0]['href']
            citation['sla_author'] = author
            citation['compact_authors'] = cpage_soup.select('div.gsc_vcd_merged_snippet div')[1].text.replace(u'\xa0', u'')
            citation['authors'] = cpage_soup.select('div.gsc_vcd_value')[0].text
            citation['date'] = cpage_soup.select('div.gsc_vcd_value')[1].text
            citation['publication'] = cpage_soup.select('div.gsc_vcd_value')[2].text
            
            time.sleep((10-5)*np.random.random()+5) #from 5 to 30 seconds

        time.sleep((10-5)*np.random.random()+5) #from 5 to 30 seconds

        author_bib.append(citation)
    return author_bib

In [580]:
cpage_soup.select('div.gsc_vcd_merged_snippet div a')[0].text

'Internet-based collaborative product design with assembly features and virtual design spaces'

In [13]:
def write_out_csv(citations):
    '''Takes a list of dictionaries and writes out as a csv file'''
    
    filename = "sla_bib.csv"
    
    if os.path.exists(filename):
        append_write = 'a' # append if already exists
    else:
        append_write = 'w' # make a new file if not
    
    with open('sla-bibliographies.csv', append_write) as sb:

        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        
        for d in cites:
            writer.writerow(d)

### run the code below
1. start webdriver, it will stay active till you driver.quit() 
  * TODO: run headless 
2. run loop that will grab uids from list

In [488]:
driver = webdriver.Firefox()

In [6]:
author_cids = []
for uid in get_users():  
    print(uid)
    page = get_full_user_page(uid)
    #show_all_articles on profile page
    author_citation_ids = get_author_citations_links(page)
    #author_cids.append(author_citation_ids)
    #write_out_csv(citations)

NameError: name 'get_users' is not defined

In [503]:
with open('outfile', 'wb') as fp:
    pickle.dump(author_cids, fp)

In [3]:
with open ('outfile', 'rb') as fp:
    authors_cid = pickle.load(fp)
print(authors_cid)

[('Rajit Gadh', ['v9rIqPQAAAAJ:u5HHmVD_uO8C', 'v9rIqPQAAAAJ:u-x6o8ySG0sC', 'v9rIqPQAAAAJ:d1gkVwhDpl0C', 'v9rIqPQAAAAJ:9yKSN-GCB0IC', 'v9rIqPQAAAAJ:2osOgNQ5qMEC', 'v9rIqPQAAAAJ:UeHWp8X0CEIC', 'v9rIqPQAAAAJ:qjMakFHDy7sC', 'v9rIqPQAAAAJ:LkGwnXOMwfcC', 'v9rIqPQAAAAJ:IjCSPb-OGe4C', 'v9rIqPQAAAAJ:zYLM7Y9cAGgC', 'v9rIqPQAAAAJ:Y0pCki6q_DkC', 'v9rIqPQAAAAJ:Tyk-4Ss8FVUC', 'v9rIqPQAAAAJ:W7OEmFMy1HYC', 'v9rIqPQAAAAJ:YsMSGLbcyi4C', 'v9rIqPQAAAAJ:eQOLeE2rZwMC', 'v9rIqPQAAAAJ:lmc2jWPfTJgC', 'v9rIqPQAAAAJ:WF5omc3nYNoC', 'v9rIqPQAAAAJ:3fE2CSJIrl8C', 'v9rIqPQAAAAJ:0EnyYjriUFMC', 'v9rIqPQAAAAJ:ufrVoPGSRksC', 'v9rIqPQAAAAJ:_FxGoFyzp5QC', 'v9rIqPQAAAAJ:Se3iqnhoufwC', 'v9rIqPQAAAAJ:UebtZRa9Y70C', 'v9rIqPQAAAAJ:roLk4NBRz8UC', 'v9rIqPQAAAAJ:5nxA0vEk-isC', 'v9rIqPQAAAAJ:hqOjcs7Dif8C', 'v9rIqPQAAAAJ:pqnbT2bcN3wC', 'v9rIqPQAAAAJ:MXK_kJrjxJIC', 'v9rIqPQAAAAJ:8k81kl-MbHgC', 'v9rIqPQAAAAJ:kNdYIx-mwKoC', 'v9rIqPQAAAAJ:Zph67rFs4hoC', 'v9rIqPQAAAAJ:KlAtU1dfN6UC', 'v9rIqPQAAAAJ:YOwf2qJgpHMC', 'v9rIqPQAAAAJ:4TOpqqG69KYC

In [501]:
driver.quit()

In [312]:
keywords_to_include = ['renewable energy', 'electric vehicles', 'battery', 'energy storage', 'EV', 'solar power',
'urban', 'North America', 'avian', 'biodiversity', 'environment', 'climate', 'solar energy', 'thermal energy', 
'electronics', 'photonics', 'batteries', 'solar', 'energy', 'urban*', 'development'] 

include_words = '''los angeles, california, environment, water
renewable, energy, algal, biofuels, biomass, fuel
climate, california, biodiversity, north america*, native
avian, biodiversity, environment, climate
transportation networks, ZEV, zero emission vehicles, public transit
thermal, battery, storage, energy, solar, wind
solar, film, transparent, power, energy, photovoltaic
aerosols, air pollution, ultrafine particles, air quality, PM2.5, PM10, ambient'''

In [213]:
import re
regex = re.compile('th.s')
l = ['this', 'is', 'just', 'a', 'test']

In [214]:
matches = [string for string in l if re.match(regex, string)]

In [6]:
r = requests.get('https://scholar.google.com/citations?user=gkygJ18AAAAJ&hl=en&oi=ao')

In [7]:
r.url

'https://scholar.google.com/citations?user=gkygJ18AAAAJ&hl=en&oi=ao'

In [10]:
csoup = BeautifulSoup(r.text, 'lxml')

In [168]:
citations = []
for cite in csoup.select('td.gsc_a_t'):
    citation = {}
    citation['title'] = cite.a.text
    citation['gchol_citation_url'] = BASE_URL+cite.a['data-href']
    citation['aid'] = cite.a['data-href'].split('&')[3].split('=')[1]
    citation['cid'] = cite.a['data-href'].split('&')[4].split('=')[1]
    citation['authors'] = cite.find_all('div')[0].text
    citation['pub'] = cite.find_all('div')[1].text
    citation['year'] = item.find_all('td')[2].text
    citation['sla_author'] = csoup.select_one('div#gsc_prf_in').text
    citations.append(citation)

In [171]:
print(pprint(citations))

[{'aid': 'gkygJ18AAAAJ',
  'authors': 'M Chester, S Pincetl, Z Elizabeth, W Eisenstein, J Matute',
  'cid': 'gkygJ18AAAAJ:roLk4NBRz8UC',
  'gchol_citation_url': 'https://scholar.google.com/citations/citations?view_op=view_citation&hl=en&oe=ASCII&user=gkygJ18AAAAJ&citation_for_view=gkygJ18AAAAJ:roLk4NBRz8UC',
  'pub': 'Environmental Research Letters 8 (1), 015041, 2013',
  'sla_author': 'Juan Matute',
  'title': 'Infrastructure and automobile shifts: positioning transit to '
           'reduce life-cycle environmental impacts for urban sustainability '
           'goals',
  'year': ''},
 {'aid': 'gkygJ18AAAAJ',
  'authors': 'M Chester, A Fraser, J Matute, C Flower, R Pendyala',
  'cid': 'gkygJ18AAAAJ:u-x6o8ySG0sC',
  'gchol_citation_url': 'https://scholar.google.com/citations/citations?view_op=view_citation&hl=en&oe=ASCII&user=gkygJ18AAAAJ&citation_for_view=gkygJ18AAAAJ:u-x6o8ySG0sC',
  'pub': 'Journal of the American Planning Association 81 (4), 268-286, 2015',
  'sla_author': 'Juan Ma